In [11]:
import pygame, sys
from pygame.locals import *
import random, time

# Initialize Pygame
pygame.init()

# Set up FPS
FPS = 60
FramePerSec = pygame.time.Clock()

# Colors
GRAY = (100, 100, 100)
GREEN = (76, 208, 56)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 232, 0)
BLACK = (0, 0, 0)

# Screen settings
SCREEN_WIDTH = 840
SCREEN_HEIGHT = 650
DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Car Game")

# Game variables
ENEMY_SPEED = 8  # Fixed speed for enemies
COIN_SPEED = ENEMY_SPEED // 2  # Coins move slower
SCORE = 0
COINS_COLLECTED = 0
RECORD = 0

# Road settings (3 lanes in center, borders as decoration)
road_width = 340  # Narrower road for 3 lanes
border_width = (SCREEN_WIDTH - road_width) // 2
lane_width = road_width // 3
road_x = border_width
lane_positions = [
    road_x + lane_width // 2,  # Left lane
    road_x + lane_width + lane_width // 2,  # Middle lane
    road_x + 2 * lane_width + lane_width // 2  # Right lane
]

# Fonts
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)
restart_text = font_small.render("Press R to Restart or Q to Quit", True, BLACK)

# Load sounds
pygame.mixer.music.load("background.wav")
pygame.mixer.music.play(-1)  # Loop background music
crash_sound = pygame.mixer.Sound("crash.wav")

# Moving lane markers
marker_width = 10
marker_height = 50
lane_marker_move_y = 0

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__() 
        self.image = pygame.image.load("taxi.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.choice(lane_positions), -50)

    def move(self):
        """Moves the enemy downward, resets position when off-screen."""
        self.rect.move_ip(0, ENEMY_SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            self.rect.center = (random.choice(lane_positions), -50)

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__() 
        self.image = pygame.image.load("car.png")
        self.rect = self.image.get_rect()
        self.current_lane = 1  # Start in the middle lane
        self.rect.center = (lane_positions[self.current_lane], 520)
    
    def move(self, direction):
        """Moves player one lane left or right when a key is pressed."""
        if direction == -1 and self.current_lane > 0:  # Move left
            self.current_lane -= 1
        elif direction == 1 and self.current_lane < 2:  # Move right
            self.current_lane += 1
        self.rect.centerx = lane_positions[self.current_lane]

class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Coin.png")
        self.image = pygame.transform.scale(self.image, (30, 30))
        self.rect = self.image.get_rect()
        self.respawn()

    def move(self):
        """Moves coin downward, resets position when off-screen."""
        self.rect.move_ip(0, COIN_SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            self.respawn()

    def respawn(self):
        """Respawns the coin at a random lane and position."""
        self.rect.center = (random.choice(lane_positions), random.randint(-100, 0))

# Reset function
def reset_game():
    """Resets game variables and recreates all objects."""
    global SCORE, COINS_COLLECTED, P1, E1, C1, all_sprites, enemies, coins, RECORD
    if SCORE > RECORD:
        RECORD = SCORE  
    SCORE = 0
    COINS_COLLECTED = 0
    P1 = Player()
    E1 = Enemy()
    C1 = Coin()
    
    coins = pygame.sprite.Group()
    coins.add(C1)
    
    enemies = pygame.sprite.Group()
    enemies.add(E1)
    
    all_sprites = pygame.sprite.Group()
    all_sprites.add(C1)
    all_sprites.add(P1)
    all_sprites.add(E1)

reset_game()

# Game loop
running = True
while running:
    direction = 0  # Default: No movement
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False
        if event.type == KEYDOWN:
            if event.key == K_r:
                reset_game()
            if event.key == K_q:
                running = False
            if event.key == K_LEFT:
                direction = -1  # Move left
            if event.key == K_RIGHT:
                direction = 1   # Move right
    
    # Background and road
    DISPLAYSURF.fill(GREEN)
    pygame.draw.rect(DISPLAYSURF, GRAY, (road_x, 0, road_width, SCREEN_HEIGHT))
    pygame.draw.rect(DISPLAYSURF, YELLOW, (road_x - 5, 0, 5, SCREEN_HEIGHT))  # Left border
    pygame.draw.rect(DISPLAYSURF, YELLOW, (road_x + road_width, 0, 5, SCREEN_HEIGHT))  # Right border

    # Moving lane markers
    lane_marker_move_y += ENEMY_SPEED * 2
    if lane_marker_move_y >= marker_height * 2:
        lane_marker_move_y = 0
    for y in range(marker_height * -2, SCREEN_HEIGHT, marker_height * 2):
        pygame.draw.rect(DISPLAYSURF, WHITE, (lane_positions[0] + lane_width // 2, y + lane_marker_move_y, marker_width, marker_height))
        pygame.draw.rect(DISPLAYSURF, WHITE, (lane_positions[1] + lane_width // 2, y + lane_marker_move_y, marker_width, marker_height))

    # Draw UI
    coins_text = font_small.render(f"Coins: {COINS_COLLECTED}", True, BLACK)
    scores = font_small.render(f"Score: {SCORE}", True, BLACK)
    record_text = font_small.render(f"Record: {RECORD}", True, BLACK)
    
    DISPLAYSURF.blit(coins_text, (10, 10))
    DISPLAYSURF.blit(scores, (SCREEN_WIDTH - 150, 10))
    DISPLAYSURF.blit(record_text, (SCREEN_WIDTH - 150, 40))
    
    # Update all entities
    for entity in all_sprites:
        if isinstance(entity, Player):
            entity.move(direction)
        else:
            entity.move()
        DISPLAYSURF.blit(entity.image, entity.rect)
    
    # Coin collection
    if pygame.sprite.spritecollideany(P1, coins):
        COINS_COLLECTED += 1
        SCORE += 1  # Score increases only when collecting a coin
        C1.respawn()
    
    # Collision detection
    if pygame.sprite.spritecollideany(P1, enemies):
        crash_sound.play()
        time.sleep(1)
        
        DISPLAYSURF.fill(RED)
        DISPLAYSURF.blit(game_over, (SCREEN_WIDTH // 2 - 150, SCREEN_HEIGHT // 2 - 30))
        DISPLAYSURF.blit(restart_text, (SCREEN_WIDTH // 2 - 150, SCREEN_HEIGHT // 2 + 30))
        pygame.display.update()
        time.sleep(2)
        reset_game()

    pygame.display.update()
    FramePerSec.tick(FPS)

pygame.quit()
sys.exit()

SystemExit: 

C:\Users\Admin\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
